In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import time

from IPython.display import Image
import torch
from collections import defaultdict
import imageio
from sradmap import SkyCoord2FisheyeUV, loadConfig
import numpy as np
from tqdm import tqdm

import math

In [2]:
source_csv_filepath = "..\..\scattered-hdr.csv"
output_csv_filepath = "..\..\coords-scattered-hdr.csv"

In [3]:
skies_data = pd.read_csv(source_csv_filepath)
skies_data.head()

,Date,Time,SunAzimuth,SunAltitude,SamplePatternIndex,SampleAzimuth,SampleAltitude,SunPointAngle,ColorA5,ColorB5,...,1771,1772,1773,1774,1775,1776,1777,1778,1779,1780
0,11/6/2012,12:32:26,191.9511,30.5111,0,0.00,12.1151,135.851,0,1,...,0.003382,0.003288,0.003172,0.002996,0.002875,0.002726,0.002590,0.002480,0.002348,0.002258
1,11/6/2012,12:32:26,191.9511,30.5111,1,348.75,12.1151,131.887,0,1,...,0.000452,0.000424,0.000429,0.000456,0.000420,0.000399,0.000385,0.000342,0.000355,0.000359
2,11/6/2012,12:32:26,191.9511,30.5111,2,337.50,12.1151,126.019,0,1,...,0.000195,0.000205,0.000198,0.000197,0.000231,0.000260,0.000254,0.000194,0.000153,0.000107
3,11/6/2012,12:32:26,191.9511,30.5111,3,326.25,12.1151,118.799,0,1,...,0.000485,0.000480,0.000445,0.000406,0.000392,0.000370,0.000339,0.000293,0.000316,0.000333
4,11/6/2012,12:32:26,191.9511,30.5111,4,315.00,12.1151,110.660,0,1,...,0.000749,0.000737,0.000745,0.000717,0.000689,0.000618,0.000597,0.000655,0.000644,0.000641


In [4]:
coordinates = skies_data[['SampleAzimuth','SampleAltitude']].values.tolist()
azimuth, altitude = coordinates[0]
azimuth, altitude

(0.0, 12.1151)

In [5]:
class Args:
    cfgpath = './config.json'
    photo = None
    timestamp = None
    cover = None
    model = None
    scaler = None
    polynomial = None
    colorbar = None

args = Args()
loadConfig(args)

def coordinates_to_uv(coord_array):
    azimuth, altitude = coord_array
    u, v = SkyCoord2FisheyeUV(args, azimuth, altitude)
    return [u, v]

uv_values = []
for coords in coordinates:
    uv_values.append(coordinates_to_uv(coords))

uv_values

[[0.5, 0.9512781234757641],
 [0.4119600055722792, 0.942606940873021],
 [0.3273033387570178, 0.9169266217493597],
 [0.24928330778392088, 0.8752240463670137],
 [0.1808981786891471, 0.819101821310853],
 [0.12477595363298621, 0.7507166922160791],
 [0.08307337825064032, 0.6726966612429823],
 [0.05739305912697906, 0.5880399944277208],
 [0.048721876524235874, 0.5],
 [0.05739305912697901, 0.41196000557227924],
 [0.0830733782506402, 0.327303338757018],
 [0.12477595363298627, 0.24928330778392088],
 [0.18089817868914698, 0.1808981786891471],
 [0.6726966612429823, 0.08307337825064037],
 [0.750716692216079, 0.12477595363298616],
 [0.8191018213108529, 0.18089817868914698],
 [0.8752240463670138, 0.24928330778392094],
 [0.9169266217493597, 0.3273033387570179],
 [0.9426069408730209, 0.41196000557227885],
 [0.9512781234757641, 0.4999999999999999],
 [0.9426069408730209, 0.5880399944277209],
 [0.9169266217493597, 0.6726966612429822],
 [0.8752240463670138, 0.7507166922160791],
 [0.819101821310853, 0.819101

In [6]:
uv_df = pd.DataFrame(uv_values)
uv_df.columns = ["u", "v"]
uv_df.head()

,u,v
0,0.500000,0.951278
1,0.411960,0.942607
2,0.327303,0.916927
3,0.249283,0.875224
4,0.180898,0.819102


In [7]:
complete_df = skies_data.join(uv_df)
complete_df.head()

,Date,Time,SunAzimuth,SunAltitude,SamplePatternIndex,SampleAzimuth,SampleAltitude,SunPointAngle,ColorA5,ColorB5,...,1773,1774,1775,1776,1777,1778,1779,1780,u,v
0,11/6/2012,12:32:26,191.9511,30.5111,0,0.00,12.1151,135.851,0,1,...,0.003172,0.002996,0.002875,0.002726,0.002590,0.002480,0.002348,0.002258,0.500000,0.951278
1,11/6/2012,12:32:26,191.9511,30.5111,1,348.75,12.1151,131.887,0,1,...,0.000429,0.000456,0.000420,0.000399,0.000385,0.000342,0.000355,0.000359,0.411960,0.942607
2,11/6/2012,12:32:26,191.9511,30.5111,2,337.50,12.1151,126.019,0,1,...,0.000198,0.000197,0.000231,0.000260,0.000254,0.000194,0.000153,0.000107,0.327303,0.916927
3,11/6/2012,12:32:26,191.9511,30.5111,3,326.25,12.1151,118.799,0,1,...,0.000445,0.000406,0.000392,0.000370,0.000339,0.000293,0.000316,0.000333,0.249283,0.875224
4,11/6/2012,12:32:26,191.9511,30.5111,4,315.00,12.1151,110.660,0,1,...,0.000745,0.000717,0.000689,0.000618,0.000597,0.000655,0.000644,0.000641,0.180898,0.819102


In [8]:
complete_df.to_csv(output_csv_filepath, index=False)